In [128]:
import numpy as np
import pandas as pd
import xarray as xr

In [129]:
dwd_Hornsea1 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc")
dwd_Hornsea1_forecast = dwd_Hornsea1["WindSpeed:100"].mean(dim=["latitude","longitude"]).to_dataframe().reset_index()
dwd_Hornsea1_forecast["ref_datetime"] = dwd_Hornsea1_forecast["ref_datetime"].dt.tz_localize("UTC")
dwd_Hornsea1_forecast["valid_datetime"] = dwd_Hornsea1_forecast["ref_datetime"] + pd.TimedeltaIndex(dwd_Hornsea1_forecast["valid_datetime"],unit="hours")
dwd_Hornsea1_forecast = dwd_Hornsea1_forecast.set_index(["ref_datetime", "valid_datetime"])

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_30027/3836930029.py:4: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  dwd_Hornsea1_forecast["valid_datetime"] = dwd_Hornsea1_forecast["ref_datetime"] + pd.TimedeltaIndex(dwd_Hornsea1_forecast["valid_datetime"],unit="hours")


In [130]:
df_coords_Hornsea1 = pd.DataFrame({'longitude': dwd_Hornsea1.longitude.values, 'latitude': dwd_Hornsea1.latitude.values, })
df_coords_Hornsea1.to_csv('data/dwd_coords_Hornsea1.csv', index=False)

In [131]:
dwd_pes10 = xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc")
dwd_pes10_forecast = dwd_pes10["SolarDownwardRadiation"].mean(dim="point").to_dataframe().reset_index()
dwd_pes10_forecast["ref_datetime"] = dwd_pes10_forecast["ref_datetime"].dt.tz_localize("UTC")
dwd_pes10_forecast["valid_datetime"] = dwd_pes10_forecast["ref_datetime"] + pd.TimedeltaIndex(dwd_pes10_forecast["valid_datetime"],unit="hours")
dwd_pes10_forecast = dwd_pes10_forecast.set_index(["ref_datetime", "valid_datetime"])

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_30027/2233848091.py:4: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  dwd_pes10_forecast["valid_datetime"] = dwd_pes10_forecast["ref_datetime"] + pd.TimedeltaIndex(dwd_pes10_forecast["valid_datetime"],unit="hours")


In [132]:
df_coords_pes10 = pd.DataFrame({'longitude': dwd_pes10.longitude.values, 'latitude': dwd_pes10.latitude.values, })
df_coords_pes10.to_csv('data/dwd_coords_solar.csv', index=False)

In [135]:
dwd_forecast = pd.concat([dwd_Hornsea1_forecast, dwd_pes10_forecast], axis=1)
dwd_forecast = dwd_forecast.rename(columns={"WindSpeed:100": "wind_forecast", "SolarDownwardRadiation": "solar_forecast"})
dwd_forecast.to_csv('data/dwd_forecast.csv')

In [136]:
dwd_forecast

wind_forecast  \
ref_datetime              valid_datetime                             
2020-09-20 00:00:00+00:00 2020-09-20 00:00:00+00:00      11.802604   
                          2020-09-20 01:00:00+00:00      11.495032   
                          2020-09-20 02:00:00+00:00      11.213223   
                          2020-09-20 03:00:00+00:00      11.135883   
                          2020-09-20 04:00:00+00:00      11.254492   
...                                                            ...   
2023-10-27 00:00:00+00:00 2023-10-31 12:00:00+00:00       3.701586   
                          2023-10-31 15:00:00+00:00       7.006474   
                          2023-10-31 18:00:00+00:00       6.105711   
                          2023-10-31 21:00:00+00:00       6.729563   
                          2023-11-01 00:00:00+00:00       5.556175   

                                                     solar_forecast  
ref_datetime              valid_datetime                             
2020-09-20 00:00:00+00:00 2020-09-20 00:00:00+00:00        0.000000  
                          2020-09-20 01:00:00+00:00        0.000000  
                          2020-09-20 02:00:00+00:00        0.000000  
                          2020-09-20 03:00:00+00:00        0.000000  
                          2020-09-20 04:00:00+00:00        0.000000  
...                                                             ...  
2023-10-27 00:00:00+00:00 2023-10-31 12:00:00+00:00      180.746119  
                          2023-10-31 15:00:00+00:00       23.697193  
                          2023-10-31 18:00:00+00:00        0.976647  
                          2023-10-31 21:00:00+00:00        0.932845  
                          2023-11-01 00:00:00+00:00             NaN  

[415431 rows x 2 columns]

In [139]:
energy_data = pd.read_csv("data/Energy_Data_20200920_20231027.csv")
energy_data["dtm"] = pd.to_datetime(energy_data["dtm"])
energy_data["wind_production"] = 0.5*energy_data["Wind_MW"] - energy_data["boa_MWh"]
energy_data["solar_production"] = 0.5*energy_data["Solar_MW"]
rename = {"dtm": "valid_datetime", 
          "Solar_installedcapacity_mwp": "solar_installed",
          "Solar_capacity_mwp": "solar_available", 
          "DA_Price": "dayahead_price", 
          "SS_Price": "imbalance_price"}
energy_data = energy_data.rename(columns=rename)
energy_data = energy_data.drop(columns=["MIP", "Wind_MW", "Solar_MW", "boa_MWh"])

#TODO Should take local time and daylight saving into account

# Function to determine the market day
def get_market_day(dt):
    # Market day changes at 23:00
    if dt.time() >= pd.Timestamp('23:00').time():
        # If the time is 23:00 or later, the market day is the next day
        return dt.date() + pd.Timedelta(days=1)
    else:
        # Otherwise, the market day is the current day
        return dt.date()

# Apply the function to the datetime column to create a new 'market_day' column
energy_data['market_day'] = energy_data['valid_datetime'].apply(get_market_day)

# Function to get specific timestamp of the day before market day
def get_decision_timestamp(market_day):
    # Subtract one day from the current market day and set the time to 09:20
    previous_day = pd.Timestamp(market_day) - pd.Timedelta(days=1)
    decision_timestamp = pd.Timestamp(previous_day.strftime('%Y-%m-%d 09:20'))
    return decision_timestamp

# Apply the function to the market_day column to create a new 'decision_timestamp' column
energy_data['decision_timestamp'] = energy_data['market_day'].apply(get_decision_timestamp)

energy_data = energy_data.iloc[:-2]
energy_data.to_csv('data/energy_data.csv', index=False)

In [140]:
energy_data

,valid_datetime,solar_available,solar_installed,imbalance_price,dayahead_price,wind_production,solar_production,market_day,decision_timestamp
0,2020-09-20 00:00:00+00:00,2108.489754,2206.064655,2.50,32.17,498.142,0.0,2020-09-20,2020-09-19 09:20:00
1,2020-09-20 00:30:00+00:00,2108.489754,2206.064655,15.00,32.17,478.788,0.0,2020-09-20,2020-09-19 09:20:00
2,2020-09-20 01:00:00+00:00,2108.489754,2206.064655,47.95,32.00,470.522,0.0,2020-09-20,2020-09-19 09:20:00
3,2020-09-20 01:30:00+00:00,2108.489754,2206.064655,29.13,32.00,482.183,0.0,2020-09-20,2020-09-19 09:20:00
4,2020-09-20 02:00:00+00:00,2108.489754,2206.064655,28.95,31.99,459.216,0.0,2020-09-20,2020-09-19 09:20:00
...,...,...,...,...,...,...,...,...,...
54377,2023-10-27 20:30:00+00:00,2331.383994,2487.180474,134.00,87.58,195.902,0.0,2023-10-27,2023-10-26 09:20:00
54378,2023-10-27 21:00:00+00:00,2331.383994,2487.180474,129.00,82.47,182.191,0.0,2023-10-27,2023-10-26 09:20:00
54379,2023-10-27 21:30:00+00:00,2331.383994,2487.180474,129.00,82.47,137.722,0.0,2023-10-27,2023-10-26 09:20:00
54380,2023-10-27 22:00:00+00:00,2331.383994,2487.180474,129.00,64.31,66.261,0.0,2023-10-27,2023-10-26 09:20:00


In [100]:
energy_data["market_day"]-pd.Timedelta(days=1)
# 1) Create a list of "decision points"
# 2) In the Environment, loop through the list and extract the available information
# 3) Build up the pandas "state variable" for each decision point

0        2020-09-19
1        2020-09-19
2        2020-09-19
3        2020-09-19
4        2020-09-19
            ...    
54377    2023-10-26
54378    2023-10-26
54379    2023-10-26
54380    2023-10-26
54381    2023-10-26
Name: market_day, Length: 54382, dtype: object

In [80]:
#Dont do this. 
data = dwd_Hornsea1_forecast.merge(dwd_pes10_forecast,how="outer",on=["ref_datetime","valid_datetime"])
data = data.set_index("valid_datetime").groupby("ref_datetime").resample("30T").interpolate("linear")
data = data.drop(columns="ref_datetime",axis=1).reset_index()
data = data.merge(energy_data,how="inner",left_on="valid_datetime",right_on="valid_datetime")
data = data[data["valid_datetime"] - data["ref_datetime"] < np.timedelta64(50,"h")]
data.rename(columns={"WindSpeed:100":"WindSpeed"}, inplace=True)
data["total_generation_MWh"] = data["Wind_MWh_credit"] + data["Solar_MWh_credit"]
data.drop(columns=["dtm"], inplace=True)

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_30027/772745781.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data = data.set_index("valid_datetime").groupby("ref_datetime").resample("30T").interpolate("linear")


KeyError: 'Wind_MWh_credit'

In [171]:
data.to_csv("data/data_heftcom2024.csv", index=False)